In [1]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice
from copy import deepcopy

from tqdm.auto import tqdm
import numpy as np

In [2]:
State = namedtuple('State', ['x', 'o'])

In [3]:
MAGIC = [2, 7, 6, 9, 5, 1, 4, 3, 8]

In [4]:
def print_board(pos):
    """Nicely prints the board"""
    for r in range(3):
        for c in range(3):
            i = r * 3 + c
            if MAGIC[i] in pos.x:
                print('X', end='')
            elif MAGIC[i] in pos.o:
                print('O', end='')
            else:
                print('.', end='')
        print()
    print()

In [5]:
def win(elements):
    """Checks is elements is winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))

def state_value(pos: State):
    """Evaluate state: +1 first player wins"""
    if win(pos.x):
        return 1
    elif win(pos.o):
        return -1
    else:
        return 0

In [6]:
def random_game():
    trajectory = list()
    state = State(set(), set())
    available = set(range(1, 9+1))
    while available:
        x = choice(list(available))
        state.x.add(x)
        trajectory.append(deepcopy(state))
        available.remove(x)
        if win(state.x) or not available:
            break

        o = choice(list(available))
        state.o.add(o)
        trajectory.append(deepcopy(state))
        available.remove(o)
        if win(state.o):
            break
    return trajectory

In [7]:
value_dictionary = defaultdict(float)
hit_state = defaultdict(int)
epsilon = 0.001

for steps in tqdm(range(500_000)):
    trajectory = random_game()
    final_reward = state_value(trajectory[-1])
    for state in trajectory:
        hashable_state = (frozenset(state.x), frozenset(state.o))
        hit_state[hashable_state] += 1
        value_dictionary[hashable_state] = value_dictionary[
            hashable_state
        ] + epsilon * (final_reward - value_dictionary[hashable_state])

  0%|          | 0/500000 [00:00<?, ?it/s]

In [8]:
sorted(value_dictionary.items(), key=lambda e: e[1], reverse=True)[:10]

[((frozenset({4, 5, 6, 8, 9}), frozenset({1, 2, 3, 7})), 0.9180176118921533),
 ((frozenset({4, 5, 6, 7, 8}), frozenset({1, 2, 3, 9})), 0.913639494211917),
 ((frozenset({2, 3, 4, 5, 6}), frozenset({1, 7, 8, 9})), 0.913639494211917),
 ((frozenset({1, 2, 3, 4, 9}), frozenset({5, 6, 7, 8})), 0.913206393246716),
 ((frozenset({1, 4, 5, 6, 7}), frozenset({2, 3, 8, 9})), 0.9128583491447205),
 ((frozenset({2, 3, 6, 7, 9}), frozenset({1, 4, 5, 8})), 0.9114521600198158),
 ((frozenset({2, 5, 6, 8, 9}), frozenset({1, 3, 4, 7})), 0.9107405789227033),
 ((frozenset({1, 2, 3, 5, 9}), frozenset({4, 6, 7, 8})), 0.9102929397956615),
 ((frozenset({1, 2, 4, 6, 7}), frozenset({3, 5, 8, 9})), 0.9099332126869561),
 ((frozenset({3, 4, 5, 6, 9}), frozenset({1, 2, 7, 8})), 0.9098430557426989)]

In [9]:
len(hit_state)

5477

In [18]:
def best_move(state, player, value_dict):
    """
    Choose the best move for the given player based on reinforcement learning.

    :param state: Actual state of the game.
    :param player: Actual player ('x' or 'o').
    :param value_dict: Dictionary of known states values.
    :return: Best move for the given player.
    """

    # Initialize the best value. For X, search the maximum, for O, search the minimum.
    best_val = -float('inf') if player == 'x' else float('inf')
    best_move = None

    # Go through all cells.
    for move in range(1, 10):

        # Checks if the cell is free.
        if move not in state.x and move not in state.o:

            # Creates a copy of the state to try the move.
            new_state = deepcopy(state)

            # If the actual player is X.
            if player == 'x':

                # Add the move to the X set.
                new_state.x.add(move)

                # Get the value of the new state from the dictionary, or 0 if not found.
                val = value_dict.get((frozenset(new_state.x), frozenset(new_state.o)), 0)

                # If the value is better than everything found up to there, update the best move.
                if val > best_val:
                    best_val = val
                    best_move = move

            # If the actual player is O, same process but with minimum value.
            else:

                new_state.o.add(move)
                val = value_dict.get((frozenset(new_state.x), frozenset(new_state.o)), 0)
                if val < best_val:
                    best_val = val
                    best_move = move

    # Return the best move found.
    return best_move

In [19]:
def rl_game(value_dict, player='x'):
    """Play a tic-tac-toe game where the given player uses reinforcement learning."""

    state = State(set(), set())
    available = set(range(1, 10))

    while available:

        if player == 'x':

            x_move = best_move(state, 'x', value_dict)
            state.x.add(x_move)
            available.remove(x_move)
            if win(state.x) or not available:
                break

            player = 'o'

        else:

            o_move = best_move(state, 'o', value_dict)
            state.o.add(o_move)
            available.remove(o_move)
            if win(state.o):
                break

            player = 'x'

    return state

In [20]:
value_dictionary = defaultdict(float)

nb_games = 10000

# Initialize the victories counter.
victoires_rl = 0

# Play n games.
for _ in range(nb_games):

    # Play a game with one player using reinforcement learning (here X).
    final_state = rl_game(value_dictionary, player='x')

    # Check if player X won.
    if win(final_state.x):
        victoires_rl += 1

# Print the number of victories for the reinforcement learning player.
print(f"The player using reinforcement learning has won {victoires_rl} times over {nb_games} games.")

The player using reinforcement learning has won 10000 times over 10000 games.
